# Assignment #3: A simple language classifier

Author: Pierre Nugues

## Objectives

In this assignment, you will implement a language detector inspired from Google's _Compact language detector_, version 3 (CLD3): https://github.com/google/cld3. CLD3 is written in C++ and its code is available from GitHub. The objectives of the assignment are to:
* Write a program to classify languages
* Use neural networks
* Know what a classifier is
* Write a short report of 1 to 2 pages to describe your program. You will notably comment the performance you obtained and how you could improve it.

## Description

### System Overview

Read the GitHub description of CLD3, https://github.com/google/cld3, (_Model_ section). In your individual report you will:
1. Summarize the system in two or three sentences;
2. Outline the CLD3 overall architecture in a figure. Use building blocks only and do not specify the parameters.

Summary: What CLD3 does is to analyze the text it gets as input and then gives back an probability along with the language that it thinks the text is from. As well as this it also gives an boolean, isReliable, that can be used to see if we can trust the output.

## Imports

In [18]:
import bz2
import json
import os
import numpy as np
import requests
import sys
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import confusion_matrix

### Dataset

As dataset, we will use Tatoeba, https://tatoeba.org/eng/downloads. It consists of more than 8 million short texts in 347 languages and it is available in one file called `sentences.csv`.

The dataset is structured this way: There is one text per line, where each line consists of the three following fields separated by tabulations and ended by a carriage return:
```
sentence id [tab] language code [tab] text [cr]
```
Each text (sentence) has a unique id and has a language code that follows the ISO 639-3 standard (see below). 

### Scope of the lab

In this lab, you will consider three languages only: French (fra), English (eng), and Swedish (swe). Below is an excerpt of the Tatoeba dataset limited to these three languages: 

```
1276    eng     Let's try something.
1277    eng     I have to go to sleep.
1280    eng     Today is June 18th and it is Muiriel's birthday!
...
1115    fra     Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.
1279    fra     Je ne supporte pas ce type.
1441    fra     Pour une fois dans ma vie je fais un bon geste... Et ça ne sert à rien.
...
337413  swe     Vi trodde att det var ett flygande tefat.
341910  swe     Detta är huset jag bodde i när jag var barn.
341938  swe     Vi hade roligt på stranden igår.
...
```
Tatoeba is updated continuously. The examples from this dataset come from a corpus your instructor downloaded on September 23, 2021.

### Understanding the $\mathbf{X}$ matrix (feature matrix)

You will now investigate the CLD3 features:
 *  What are the features CLD3 extracts from each text?
 * Create manually a simplified $\mathbf{X}$ matrix where you will represent the 9 texts with CLD3 features. You will use a restricted set of features: You will only consider the letters _a_, _b_, and _n_ and the bigrams _an_, _ba_, and _na_. You will ignore the the rest of letters and bigrams as well as the trigrams. Your matrix will have 9 rows and 6 columns, each column will contain these counts: `[#a, #b, #n, #an, #ba, #na]`.

The CLD3's original description uses relative frequencies (counts of a letter divided by the total counts of letters in the text). Here, you will use the raw counts. To help you, your instructor filled the fourth row of the matrix corresponding to the first text in French. Fill in the rest. You will include this matrix in your report. 

$\mathbf{X} =
\begin{bmatrix}
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
8& 0& 8& 1&0&0\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
     \text{eng} \\
     \text{eng}\\
     \text{eng}\\
    \text{fra}\\
   \text{fra}  \\
     \text{fra}\\
    \text{swe}\\
 \text{swe}   \\
 \text{swe}   
\end{bmatrix}$

## Programming: Extracting the features

Before you start programming, download the Tatoeba dataset.

### Loading and filtering the dataset

Run the code to read the dataset and split it into lines. You may have to change the path

In [19]:
dataset = open('sentences.csv', encoding='utf8').read().strip()
dataset = dataset.split('\n')
dataset[:10]

['1\tcmn\t我們試試看！',
 '2\tcmn\t我该去睡觉了。',
 '3\tcmn\t你在干什麼啊？',
 '4\tcmn\t這是什麼啊？',
 '5\tcmn\t今天是６月１８号，也是Muiriel的生日！',
 '6\tcmn\t生日快乐，Muiriel！',
 '7\tcmn\tMuiriel现在20岁了。',
 '8\tcmn\t密码是"Muiriel"。',
 '9\tcmn\t我很快就會回來。',
 '10\tcmn\t我不知道。']

Run the code to split the fields and remove possible whitespaces

In [20]:
dataset = list(map(lambda x: tuple(x.split('\t')), dataset))
dataset = list(map(lambda x: tuple(map(str.strip, x)), dataset))
dataset[:3]

[('1', 'cmn', '我們試試看！'), ('2', 'cmn', '我该去睡觉了。'), ('3', 'cmn', '你在干什麼啊？')]

Write the code to extract the French, English, and Swedish texts. You will call the resulting dataset: `dataset_small`

In [21]:
# Write your code here List((ID, LANG, SENTENCE)
dataset_small = [line for line in dataset if line[1] == 'eng' or line[1] == 'swe' or line[1] == 'fra']

In [22]:
dataset_small[:5]

[('1115',
  'fra',
  "Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent."),
 ('1276', 'eng', "Let's try something."),
 ('1277', 'eng', 'I have to go to sleep.'),
 ('1279', 'fra', 'Je ne supporte pas ce type.'),
 ('1280', 'eng', "Today is June 18th and it is Muiriel's birthday!")]

### Functions to Count Characters Ngrams

Write a function `count_chars(string, lc=True)` to count characters (unigrams) of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the unigrams.

In [23]:
# Write your code here
# TODO fix so we have relative frequencies on unigrams, bigrams, trigrams.
def count_chars(string, lc=True):
    tmp = string
    if lc:
        tmp = tmp.lower()
    
    total = len(tmp)
    dic = {}
    for char in tmp:
        try:
            dic[char] += 1
        except KeyError:
            dic[char] = 1
    
    for value in dic:
        tmp = dic[value] 
        tmp = tmp / total
        dic[value] = tmp
            
    return dic
count_chars("Sladjköfklöasdfafasdklasdf", lc=False)
#With lowercase {'s': 4, 'l': 3, 'a': 5, 'd': 4, 'j': 1, 'k': 3, 'ö': 2, 'f': 4}

{'S': 0.038461538461538464,
 'l': 0.11538461538461539,
 'a': 0.19230769230769232,
 'd': 0.15384615384615385,
 'j': 0.038461538461538464,
 'k': 0.11538461538461539,
 'ö': 0.07692307692307693,
 'f': 0.15384615384615385,
 's': 0.11538461538461539}

Write a function `count_bigrams(string, lc=True)` to count the characters bigrams of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the bigrams.

In [24]:
# Write your code here
def count_bigrams(string, lc=True): # Type should be Dict((c,e): 1)
    tmp = string
    if lc:
        tmp = tmp.lower()
    
    total = len(tmp) -1

    bi_dic = {}
    for i in range(len(tmp) - 1):
        try:
            bi_dic[tmp[i] + tmp[i + 1]] += 1
        except KeyError:
            bi_dic[tmp[i] + tmp[i + 1]] = 1
            
    for value in bi_dic:
        tmp = bi_dic[value] 
        tmp = tmp / total
        bi_dic[value] = tmp
    
    return bi_dic
count_bigrams("SSSsssSSSSladjköfklöaaasdfafasdklasdf", lc=False)

{'SS': 0.1388888888888889,
 'Ss': 0.027777777777777776,
 'ss': 0.05555555555555555,
 'sS': 0.027777777777777776,
 'Sl': 0.027777777777777776,
 'la': 0.05555555555555555,
 'ad': 0.027777777777777776,
 'dj': 0.027777777777777776,
 'jk': 0.027777777777777776,
 'kö': 0.027777777777777776,
 'öf': 0.027777777777777776,
 'fk': 0.027777777777777776,
 'kl': 0.05555555555555555,
 'lö': 0.027777777777777776,
 'öa': 0.027777777777777776,
 'aa': 0.05555555555555555,
 'as': 0.08333333333333333,
 'sd': 0.08333333333333333,
 'df': 0.05555555555555555,
 'fa': 0.05555555555555555,
 'af': 0.027777777777777776,
 'dk': 0.027777777777777776}

Write a function `count_trigrams(string, lc=True)` to count the characters trigrams of a string. You will set the text in lowercase if `lc` is set to `True`. As in CLD3, you will return the relative frequencies of the trigrams.

In [25]:
# Write your code here
def count_trigrams(string, lc=True): # Type should be Dict((c,e): 1)
    tmp = string
    if lc:
        tmp = tmp.lower()
    
    total = len(tmp) - 2 
    
    tri_dic = {}
    for i in range(len(tmp) - 2):
        try:
            tri_dic[tmp[i] + tmp[i + 1] + tmp[i + 2]] += 1
        except KeyError:
            tri_dic[tmp[i] + tmp[i + 1] + tmp[i + 2]] = 1
    
    for value in tri_dic:
        tmp = tri_dic[value] 
        tmp = tmp / total
        tri_dic[value] = tmp
                
    return tri_dic
count_trigrams("SSSsssSSSSladjköfklöaaasdfafasdklasdf", lc=False)

{'SSS': 0.08571428571428572,
 'SSs': 0.02857142857142857,
 'Sss': 0.02857142857142857,
 'sss': 0.02857142857142857,
 'ssS': 0.02857142857142857,
 'sSS': 0.02857142857142857,
 'SSl': 0.02857142857142857,
 'Sla': 0.02857142857142857,
 'lad': 0.02857142857142857,
 'adj': 0.02857142857142857,
 'djk': 0.02857142857142857,
 'jkö': 0.02857142857142857,
 'köf': 0.02857142857142857,
 'öfk': 0.02857142857142857,
 'fkl': 0.02857142857142857,
 'klö': 0.02857142857142857,
 'löa': 0.02857142857142857,
 'öaa': 0.02857142857142857,
 'aaa': 0.02857142857142857,
 'aas': 0.02857142857142857,
 'asd': 0.08571428571428572,
 'sdf': 0.05714285714285714,
 'dfa': 0.02857142857142857,
 'faf': 0.02857142857142857,
 'afa': 0.02857142857142857,
 'fas': 0.02857142857142857,
 'sdk': 0.02857142857142857,
 'dkl': 0.02857142857142857,
 'kla': 0.02857142857142857,
 'las': 0.02857142857142857}

In [26]:
count_chars("Let's try something.")

{'l': 0.05,
 'e': 0.1,
 't': 0.15,
 "'": 0.05,
 's': 0.1,
 ' ': 0.1,
 'r': 0.05,
 'y': 0.05,
 'o': 0.05,
 'm': 0.05,
 'h': 0.05,
 'i': 0.05,
 'n': 0.05,
 'g': 0.05,
 '.': 0.05}

In [27]:
count_chars("Let's try something.")

{'l': 0.05,
 'e': 0.1,
 't': 0.15,
 "'": 0.05,
 's': 0.1,
 ' ': 0.1,
 'r': 0.05,
 'y': 0.05,
 'o': 0.05,
 'm': 0.05,
 'h': 0.05,
 'i': 0.05,
 'n': 0.05,
 'g': 0.05,
 '.': 0.05}

In [28]:
count_bigrams("Let's try something.")

{'le': 0.05263157894736842,
 'et': 0.10526315789473684,
 "t'": 0.05263157894736842,
 "'s": 0.05263157894736842,
 's ': 0.05263157894736842,
 ' t': 0.05263157894736842,
 'tr': 0.05263157894736842,
 'ry': 0.05263157894736842,
 'y ': 0.05263157894736842,
 ' s': 0.05263157894736842,
 'so': 0.05263157894736842,
 'om': 0.05263157894736842,
 'me': 0.05263157894736842,
 'th': 0.05263157894736842,
 'hi': 0.05263157894736842,
 'in': 0.05263157894736842,
 'ng': 0.05263157894736842,
 'g.': 0.05263157894736842}

In [29]:
count_bigrams("Let's try something.")

{'le': 0.05263157894736842,
 'et': 0.10526315789473684,
 "t'": 0.05263157894736842,
 "'s": 0.05263157894736842,
 's ': 0.05263157894736842,
 ' t': 0.05263157894736842,
 'tr': 0.05263157894736842,
 'ry': 0.05263157894736842,
 'y ': 0.05263157894736842,
 ' s': 0.05263157894736842,
 'so': 0.05263157894736842,
 'om': 0.05263157894736842,
 'me': 0.05263157894736842,
 'th': 0.05263157894736842,
 'hi': 0.05263157894736842,
 'in': 0.05263157894736842,
 'ng': 0.05263157894736842,
 'g.': 0.05263157894736842}

In [30]:
count_trigrams("Let's try something.")

{'let': 0.05555555555555555,
 "et'": 0.05555555555555555,
 "t's": 0.05555555555555555,
 "'s ": 0.05555555555555555,
 's t': 0.05555555555555555,
 ' tr': 0.05555555555555555,
 'try': 0.05555555555555555,
 'ry ': 0.05555555555555555,
 'y s': 0.05555555555555555,
 ' so': 0.05555555555555555,
 'som': 0.05555555555555555,
 'ome': 0.05555555555555555,
 'met': 0.05555555555555555,
 'eth': 0.05555555555555555,
 'thi': 0.05555555555555555,
 'hin': 0.05555555555555555,
 'ing': 0.05555555555555555,
 'ng.': 0.05555555555555555}

In [31]:
count_trigrams("Let's try something.")

{'let': 0.05555555555555555,
 "et'": 0.05555555555555555,
 "t's": 0.05555555555555555,
 "'s ": 0.05555555555555555,
 's t': 0.05555555555555555,
 ' tr': 0.05555555555555555,
 'try': 0.05555555555555555,
 'ry ': 0.05555555555555555,
 'y s': 0.05555555555555555,
 ' so': 0.05555555555555555,
 'som': 0.05555555555555555,
 'ome': 0.05555555555555555,
 'met': 0.05555555555555555,
 'eth': 0.05555555555555555,
 'thi': 0.05555555555555555,
 'hin': 0.05555555555555555,
 'ing': 0.05555555555555555,
 'ng.': 0.05555555555555555}

### Counting the ngrams in the dataset

You will now extract the features from each text. For this, add the character, bigram, and trigram relative frequencies to the texts using this format:
`(text_id, language_id, text, char_cnt, bigram_cnt, trigram_cnt)`.

From the datapoint:
`('1276', 'eng', "Let's try something.")`,
you must return:

`('1276', 'eng', "Let's try something.", 
  {'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 'n': 0.05, 'g': 0.05, '.': 0.05},
  {'le': 0.05263157894736842, 'et': 0.10526315789473684, "t'": 0.05263157894736842, "'s": 0.05263157894736842, 's ': 0.05263157894736842, ' t': 0.05263157894736842, 'tr': 0.05263157894736842, 'ry': 0.05263157894736842, 'y ': 0.05263157894736842, ' s': 0.05263157894736842, 'so': 0.05263157894736842, 'om': 0.05263157894736842, 'me': 0.05263157894736842, 'th': 0.05263157894736842, 'hi': 0.05263157894736842, 'in': 0.05263157894736842, 'ng': 0.05263157894736842, 'g.': 0.05263157894736842},
  {'let': 0.05555555555555555, "et'": 0.05555555555555555, "t's": 0.05555555555555555, "'s ": 0.05555555555555555, 's t': 0.05555555555555555, ' tr': 0.05555555555555555, 'try': 0.05555555555555555, 'ry ': 0.05555555555555555, 'y s': 0.05555555555555555, ' so': 0.05555555555555555, 'som': 0.05555555555555555, 'ome': 0.05555555555555555, 'met': 0.05555555555555555, 'eth': 0.05555555555555555, 'thi': 0.05555555555555555, 'hin': 0.05555555555555555, 'ing': 0.05555555555555555, 'ng.': 0.05555555555555555})`

You will store the extracted features in a list that you will call `dataset_small_feat`

We only compute the unigrams and bigrams as most students have slow machines

In [ ]:
# Write your code here
dataset_small_feat = [(line[0], line[1], line[2], count_chars(line[2]), count_bigrams(line[2])) for line in dataset_small ]

In [ ]:
dataset_small_feat[:2]

In [ ]:
dataset_small_feat[:2]

The unigram frequencies

In [ ]:
dataset_small_feat[0][3].items()

In [ ]:
dataset_small_feat[0][3].items()

The bigram frequencies

In [ ]:
dataset_small_feat[0][4].items()

In [ ]:
dataset_small_feat[0][4].items()

## Programming: Building $\mathbf{X}$

You will now build the $\mathbf{X}$ matrix. In this assignment, you will only consider unigrams to speed up the training step. This means that you will set aside the character bigrams and trigrams.

When you are done with the lab requirements, feel free to improve the program and include bigrams and trigrams. To add bigrams, a possible method is to add the bigram dictionary to the unigram one using update and then to extract the resulting dictionary. You can easily extend this to trigrams. Feel free to use another method if you want.

In [ ]:
INCLUDE_BIGRAMS = False
if INCLUDE_BIGRAMS:
    for i in range(len(dataset_small_feat)):
        dataset_small_feat[i][3].update(dataset_small_feat[i][4])

### Vectorizing the features

The CLD3 architecture uses embeddings. In this lab, we will simplify it and we will use a feature vector instead consisting of the character frequencies. For example, you will represent the text:

`"Let's try something."`

with:

`{'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 
 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 
 'n': 0.05, 'g': 0.05, '.': 0.05}`

To create the $\mathbf{X}$ matrix, we need to transform the dictionaries of `dataset_small` into numerical vectors. The `DictVectorizer` class from the scikit-learn library, see here [https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html], has two methods, `fit()` and `transform()`, and a combination of both `fit_transform()` to convert dictionaries into such vectors.

You will now write the code to:

1. Extract the character frequency dictionaries from `dataset_small` corresponding to its 3rd index and set them in a list;
2. Convert the list of dictionaries into an $\mathbf{X}$ matrix using `DictVectorizer`.

#### Extracting the character frequencies

Produce a new list of datapoints with the unigrams only. Each item in this list will be a dictionary. You will call it `X_cat`

In [ ]:
X_cat = [count_chars(line[2]) for line in dataset_small]

In [ ]:
X_cat[:2]

In [ ]:
X_cat[:2]

#### Vectorize `X_cat`

Convert you `X_cat` matrix into a numerical representation using `DictVectorizer`: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html

In [ ]:
# Write your code here
v = DictVectorizer(sparse = False)
X = v.fit_transform(X_cat)



In [ ]:
X[:2]

## Programming: Building $\mathbf{y}$

You will now convert the list of language symbols into a $\mathbf{y}$ vector

Extract the language symbols from `dataset_small_feat` and call the resulting list `y_cat`

In [ ]:
# Write your code here
y_cat = [word[1] for word in dataset_small_feat]

In [ ]:
y_cat[:5]

In [ ]:
y_cat[:5]

Extract the set of language symbols and build two indices mapping the symbols to integers and the integers to symbols. Both indices will be dictionaries that you will call: `lang2inx`and `inx2lang`. Such a conversion is not necessary with sklearn. We do it because many other many machine-learning toolkits (keras or pytorch) require a numerical $\mathbf{y}$ vector and to learn how to carry out this conversion.

In [ ]:
# Write your code here
y_symbols = set(y_cat)
inx2lang = dict(enumerate(y_symbols))
lang2inx = {v: k for k,v in inx2lang.items()}

In [ ]:
inx2lang

In [ ]:
inx2lang

In [ ]:
lang2inx

In [ ]:
lang2inx

Convert your `y_cat` vector into a numerical vector. Call this vector `y`.

In [ ]:
# Write your code here
y = [lang for lang in y_cat]



In [ ]:
y[:5]

## Programming: Building the Model

Create a neural network using sklearn with a hidden layer of 50 nodes and a relu activation layer: https://scikit-learn.org/stable/modules/neural_networks_supervised.html. Set the maximal number of iterations to 5, in the beginning, and verbose to True. Use the default values for the rest. You will call your classifier `clf`

In [ ]:
# Write your code here
clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter = 5, verbose = True)

In [ ]:
clf

### Training and Validation Sets

You will now split the dataset into a training and validation sets

#### We shuffle the indices

In [ ]:
indices = list(range(X.shape[0]))
np.random.shuffle(indices)
print(indices[:10])
X = X[indices, :]
y = np.array(y)[indices]

#### We split the dataset

In [ ]:
training_examples = int(X.shape[0] * 0.8)

X_train = X[:training_examples, :]
y_train = y[:training_examples]

X_val = X[training_examples:, :]
y_val = y[training_examples:]

### Fitting the model

Fit the model on the training set

In [ ]:
# Write your code here
clf.fit(X, y)

## Predicting

Predict the `X_val` languages. You will call the result `y_val_pred`

In [ ]:
# Write your code here


In [ ]:
print(y_val_pred[:10])
print(y_val[:10])

#### Evaluating

In [ ]:
# evaluate the model
accuracy_score(y_val, y_val_pred)

In [ ]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

### Confusion Matrix

In [ ]:
confusion_matrix(y_val, y_val_pred)

Increase the number of iterations to improve the score. You may also change the parameters.

## Predict the language of a text

Now predict the languages of the strings below.

In [ ]:
docs = ["Salut les gars !", "Hejsan grabbar!", "Hello guys!", "Hejsan tjejer!"]

Create features vectors from this list. Call this matrix `X_test`

In [ ]:
# Write your code here

In [ ]:
X_test

And run the prediction that you will store in a variable called `pred_languages`

In [ ]:
# Write your code here

In [ ]:
pred_languages

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [ ]:
STIL_ID = ["student1", "student1"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "3-language_detector_solution.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the predicted languages.

In [ ]:
ANSWER = json.dumps({'pred_langs': pred_languages})
ANSWER

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [ ]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [ ]:
ASSIGNMENT = 3
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [ ]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
               verify=True)

# from IPython.display import display, JSON
res.json()

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to:
   * Summarize CLD3 and outline its architecture
   * Identify the features used by CLD3
   * Include the feature matrix you computed manually

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is October 1st, 2021.

## Postscript from Pierre Nugues

I created this assignment from an examination I wrote in 2019 for the course on applied machine learning. I simplified it from the `README.md` on GitHub, https://github.com/google/cld3. I found the C++ code difficult to understand and I reimplemented a Keras/Tensorflow version of it from this `README`. Should you be interested, you can find it here: https://github.com/pnugues/language-detector.